# Ligand modification with RDKit

This notebook demonstrates loading and modifying a molecule with RDKit.

## OpenFF Code

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

orig_ligand = Molecule.from_file("inputs/PT2385.sdf")
orig_top = Topology.from_pdb(
    "inputs/solvated_complex.pdb", unique_molecules=[orig_ligand]
)
ff = ForceField("openff-2.0.0.offxml", "ff14sb_off_impropers_0.0.3.offxml")

In [ ]:
orig_ligand.visualize(backend="rdkit")

## RDKit Code

In [ ]:
# Use RDKit's reaction handling to try mutating each aliphatic C-H bond to C-F
import rdkit

rdmol = orig_ligand.to_rdkit()
rxn = rdkit.Chem.rdChemReactions.ReactionFromSmarts("[C:1][H:2] >> [C:1][F:2]")
unsanitized_products = rxn.RunReactants([rdmol])

# Since some substitutions won't actually lead to a "new" molecule, only keep
# unique molecules
products = set()
for (product,) in unsanitized_products:
    mol_copy = rdkit.Chem.Mol(product)
    
    rdkit.Chem.SanitizeMol(mol_copy, rdkit.Chem.SANITIZE_ALL)
    rdkit.Chem.AssignStereochemistryFrom3D(mol_copy)
    rdkit.Chem.Kekulize(mol_copy, clearAromaticFlags=True)
    rdkit.Chem.SetAromaticity(mol_copy, rdkit.Chem.AromaticityModel.AROMATICITY_MDL)
    
    products.add(Molecule.from_rdkit(mol_copy))

In [ ]:
def display_molecule_grid(molecules, item_width=200):
    import ipywidgets as widgets
    
    items = []
    for product in products:
        item = widgets.Output()
        item.append_display_data(product.visualize('rdkit', width=item_width))
        items.append(item)
        
    return widgets.GridBox(
        items, 
        layout=widgets.Layout(grid_template_columns=f"repeat(3, {item_width+10}px)")
    )
display_molecule_grid(products)

## OpenFF and OpenMM Code

In [ ]:
import openmm
from openff.units import Quantity, unit
from openmm import unit as openmm_unit

idx_to_sim = dict()
n_systems = len(products)

for idx, unique_offmol in enumerate(products):
    print(f"Making topology {idx+1}/{n_systems}")

    # Create a new topology from the with all but the last molecule (the ligand)
    this_top = Topology.from_molecules([*orig_top.molecules][:-1] + [unique_offmol])
    this_top.box_vectors = orig_top.box_vectors
    pdb_filename = f"topology{idx+1}.pdb"
    this_top.to_file(pdb_filename)

    print(f"Parametrizing system {idx+1}/{n_systems}")
    integrator = openmm.LangevinIntegrator(
        300 * openmm_unit.kelvin,
        1 / openmm_unit.picosecond,
        0.002 * openmm_unit.picoseconds,
    )
    
    simulation = ff.create_interchange(this_top).to_openmm_simulation(integrator)

    print(f"Minimizing system {idx+1}/{n_systems}")
    simulation.minimizeEnergy()
    
    print(f"Simulating system {idx+1}/{n_systems}")
    traj_filename = f"trajectory_{idx+1}.dcd"
    dcd_reporter = openmm.app.DCDReporter(traj_filename, 50)
    simulation.reporters.append(dcd_reporter)
    simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
    simulation.runForClockTime(15 * openmm_unit.second)
    idx_to_sim[idx + 1] = {
        "topology": this_top,
        "pdb_filename": pdb_filename,
        "simulation": simulation,
        "trajectory": traj_filename,
    }

## NGLView

In [ ]:
import mdtraj
import nglview

SIM_TO_VISUALIZE = 1

trajectory: mdtraj.Trajectory = mdtraj.load(
    idx_to_sim[SIM_TO_VISUALIZE]["trajectory"],
    top=mdtraj.load(idx_to_sim[SIM_TO_VISUALIZE]["pdb_filename"]).topology,
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="water")
view.add_representation(
    "hyperball", radiusSize=1, radiusScale=0.5, selection="not protein and not water"
)
view